In [1]:
"""
dollar cost averaging comparison

v1.0

v1.0 successfully uses DCA

The goal of this sheet is to make a method that compares dollar cost averaging of different indexes and stocks
    
after i create the rules that dictate how indexes are bought I'm going to look for alternatives to DCA including
looking at investing during statistical biweekly, and monthly lows

"""

"\ndollar cost averaging comparison\n\nv1.0\n\nThe goal of this sheet is to make a method that compares dollar cost averaging of different indexes and stocks\n    \nafter i create the rules that dictate how indexes are bought I'm going to look for alternatives to DCA including\nlooking at investing during statistical biweekly, and monthly lows\n\n"

In [2]:
import yfinance as yf
import pandas as pd
import numpy as np
import math

In [3]:
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

%matplotlib inline

In [4]:
# stock methods

def get_latest_price(ticker):
    return yf.Ticker(ticker).history(period='1d').iloc[0]['Close']

def get_last_day(ticker):
    return yf.Ticker(ticker).history(period='1d').reset_index().iloc[0]['Date']

In [5]:
class create_tradebot():
    
    def __init__(self, principal, payment, name):
        self.holdings = {'$': principal}
        self.payment = payment
        self.totalCashInput = principal
        self.name = name
        
        
    def getCash(self): 
        return self.holdings['$']
    
    
    def addCash(self, amount):
        self.totalCashInput += amount
        self.holdings['$'] += amount
    
    
    def buy(self, ticker, amount, date):
            
        df = yf.Ticker(ticker).history(period='max').sort_values(by=['Date'], ascending=False).reset_index()
        openPrice = df[df['Date']==date].iloc[0]['Open']
        
        if ticker in self.holdings:
            self.holdings[ticker] = self.holdings[ticker] + math.floor(amount / openPrice)
            
            self.holdings['$'] = round(amount % openPrice,2)

        else:
            self.holdings.update(
                {ticker: math.floor(amount / openPrice)})

            self.holdings['$'] = round(amount % openPrice,2)
        
        
    def get_current_value(self):
        
        holdings_list= [key for key,units in self.holdings.items()]
        
        value = self.holdings['$']
        
        for key in holdings_list[1:]:
            value = value + (self.holdings[key] * get_latest_price(key))
            
        return value
        
        
    def dollarCostAveraging(self, ticker, amount, date):
#         gets the entire history
        df = yf.Ticker(ticker).history(start=date).sort_values(by=['Date'], ascending=False).reset_index()
    
#     gets every 10th row mimicking a paycheck every 2 weeks (5 trading days / week)
        paycheck_interval_df = df.iloc[::10, :].reset_index().drop(columns=['index'])
        
        for index, row in paycheck_interval_df.iterrows():
            self.addCash(self.payment)
            self.buy(ticker, self.holdings['$'], row.Date)

In [6]:
"""
Put the below tickers into a list of dictionaries so i can scale up on individual stocks
    or create an object where the name points to the characteristics I'm looking for

check syntax
"""

main_stocks = [
    {'Name': 'sp500', 'Ticker':'^GSPC', 'Category':'index'},
    {'Name': 'vanguard500', 'Ticker': 'VOO', 'Category':'tracker'},
    {'Name': 'nasdaq100', 'Ticker': 'QQQ', 'Category':'tracker'},
    {'Name': 'DA_sp1500', 'Ticker':'SDY', 'Category':'dividend aristocrat'},
    {'Name': 'DA_nasdaq', 'Ticker':'VIG', 'Category':'dividend aristocrat'},
    {'Name': 'BC_Innovation', 'Ticker':'KOIN', 'Category':'BC Benefactor'},
    {'Name': 'QC_Bluestar', 'Ticker': 'QTUM', 'Category': 'index'}
]

alt_stocks = [
    {'Name': 'nasdaq', 'Ticker': '^IXIC', 'Category':'index'},
    {'Name': 'vanguardTotal', 'Ticker': 'VTSAX', 'Category':'tracker'},
    {'Name': 'AmericanLTG', 'Ticker':'AIVSX', 'Category':'tracker'},
    {'Name': 'ValueHunting', 'Ticker':'PIODX', 'Category':'tracker'}, 
    {'Name': 'DA_DowJones', 'Ticker':'DVY', 'Category':'dividend aristocrat'},
    {'Name': 'DA_PreferredShares', 'Ticker':'PFF', 'Category':'dividend aristocrat'},
    {'Name': 'DA_WeightedByDividend', 'Ticker':'DTD', 'Category':'dividend aristocrat'},
    {'Name': 'BC_Economy', 'Ticker':'BLCN', 'Category':'BC Benefactor'},
    {'Name': 'BC_WW', 'Ticker':'LEGR', 'Category':'BC Benefactor'}
]

stocks = main_stocks
stocks.extend(alt_stocks)


In [7]:
for i, n in enumerate([d['Ticker'] for d in stocks if 'Ticker' in d]):
    print(i)
    df = yf.Ticker(n).history(period='max').reset_index()
    df['PriorYearBW'] = df.Open/df.Open.shift(253)-1
    stocks[i].update({"Average":df['PriorYearBW'].mean(), "Risk":df['PriorYearBW'].std(),
                      "GrowthRiskRatio":df['PriorYearBW'].mean()/df['PriorYearBW'].std(),
                    "StartDate":df['Date'].iloc[0], "EndDate":df['Date'].iloc[-1]})
    
    stocks[i].update(yf.Ticker(n).info)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15


In [8]:
summary_df = pd.DataFrame(stocks)
summary_df = summary_df.sort_values(by=['StartDate'], ascending=True)
summary_df

,Name,Ticker,Category,Average,Risk,GrowthRiskRatio,StartDate,EndDate,previousClose,regularMarketOpen,...,lastCapGain,category,fiveYearAverageReturn,phone,longBusinessSummary,companyOfficers,trailingPE,address1,address2,address3
0,sp500,^GSPC,index,0.077137,0.202079,0.381719,1927-12-30,2020-06-03,3080.8200,3098.90,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,nasdaq,^IXIC,index,0.122134,0.237656,0.513910,1971-02-05,2020-06-03,9608.3800,9651.86,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,ValueHunting,PIODX,tracker,0.091264,0.158792,0.574738,1973-05-03,2020-06-03,29.3400,NaN,...,0.0,None,NaN,800-225-6292,The investment seeks reasonable income and cap...,[],NaN,Pioneer Fund,"60 State St., 13th Floor","Boston, MA 2109"
9,AmericanLTG,AIVSX,tracker,0.097387,0.136272,0.714652,1986-01-02,2020-06-03,37.7200,NaN,...,0.0,None,NaN,800-421-4225,The investment seeks long-term growth of capit...,[],NaN,THE INVESTMENT COMPANY OF AMERICA,333 South Hope Street,"Los Angeles, CA 90071-1406"
2,nasdaq100,QQQ,tracker,0.103932,0.269728,0.385322,1999-03-10,2020-06-03,235.6300,236.23,...,NaN,Large Growth,0.1760,800-983-0903,The investment seeks investment results that g...,[],60.767650,NaN,NaN,NaN
8,vanguardTotal,VTSAX,tracker,0.090942,0.167961,0.541446,2000-11-13,2020-06-03,75.9200,NaN,...,0.0,None,NaN,800-662-7447,The investment seeks to track the performance ...,[],10.383993,Vanguard Index Funds,PO Box 2600,"Valley Forge, PA 19482"
11,DA_DowJones,DVY,dividend aristocrat,0.087601,0.162436,0.539297,2003-11-07,2020-06-03,84.2200,84.96,...,NaN,None,0.0244,800-474-2737,The investment seeks to track the investment r...,[],8.919673,NaN,NaN,NaN
3,DA_sp1500,SDY,dividend aristocrat,0.095249,0.148044,0.643386,2005-11-15,2020-06-03,92.4000,93.53,...,NaN,Large Value,0.0811,866-787-2257,The investment seeks to provide investment res...,[],NaN,NaN,NaN,NaN
4,DA_nasdaq,VIG,dividend aristocrat,0.096682,0.135713,0.712404,2006-05-02,2020-06-03,118.7800,119.61,...,NaN,Large Blend,0.1039,800-662-7447,The investment seeks to track the performance ...,[],NaN,NaN,NaN,NaN
13,DA_WeightedByDividend,DTD,dividend aristocrat,0.087489,0.165597,0.528327,2006-06-16,2020-06-03,90.9900,91.62,...,NaN,Large Value,0.0725,866-909-9473,The investment seeks to track the price and yi...,[],NaN,NaN,NaN,NaN


In [9]:
begin = '2010-09-09'
filtered_df = summary_df[summary_df['StartDate']<=begin]
filtered_df

,Name,Ticker,Category,Average,Risk,GrowthRiskRatio,StartDate,EndDate,previousClose,regularMarketOpen,...,lastCapGain,category,fiveYearAverageReturn,phone,longBusinessSummary,companyOfficers,trailingPE,address1,address2,address3
0,sp500,^GSPC,index,0.077137,0.202079,0.381719,1927-12-30,2020-06-03,3080.82,3098.90,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,nasdaq,^IXIC,index,0.122134,0.237656,0.513910,1971-02-05,2020-06-03,9608.38,9651.86,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,ValueHunting,PIODX,tracker,0.091264,0.158792,0.574738,1973-05-03,2020-06-03,29.34,NaN,...,0.0,None,NaN,800-225-6292,The investment seeks reasonable income and cap...,[],NaN,Pioneer Fund,"60 State St., 13th Floor","Boston, MA 2109"
9,AmericanLTG,AIVSX,tracker,0.097387,0.136272,0.714652,1986-01-02,2020-06-03,37.72,NaN,...,0.0,None,NaN,800-421-4225,The investment seeks long-term growth of capit...,[],NaN,THE INVESTMENT COMPANY OF AMERICA,333 South Hope Street,"Los Angeles, CA 90071-1406"
2,nasdaq100,QQQ,tracker,0.103932,0.269728,0.385322,1999-03-10,2020-06-03,235.63,236.23,...,NaN,Large Growth,0.1760,800-983-0903,The investment seeks investment results that g...,[],60.767650,NaN,NaN,NaN
8,vanguardTotal,VTSAX,tracker,0.090942,0.167961,0.541446,2000-11-13,2020-06-03,75.92,NaN,...,0.0,None,NaN,800-662-7447,The investment seeks to track the performance ...,[],10.383993,Vanguard Index Funds,PO Box 2600,"Valley Forge, PA 19482"
11,DA_DowJones,DVY,dividend aristocrat,0.087601,0.162436,0.539297,2003-11-07,2020-06-03,84.22,84.96,...,NaN,None,0.0244,800-474-2737,The investment seeks to track the investment r...,[],8.919673,NaN,NaN,NaN
3,DA_sp1500,SDY,dividend aristocrat,0.095249,0.148044,0.643386,2005-11-15,2020-06-03,92.40,93.53,...,NaN,Large Value,0.0811,866-787-2257,The investment seeks to provide investment res...,[],NaN,NaN,NaN,NaN
4,DA_nasdaq,VIG,dividend aristocrat,0.096682,0.135713,0.712404,2006-05-02,2020-06-03,118.78,119.61,...,NaN,Large Blend,0.1039,800-662-7447,The investment seeks to track the performance ...,[],NaN,NaN,NaN,NaN
13,DA_WeightedByDividend,DTD,dividend aristocrat,0.087489,0.165597,0.528327,2006-06-16,2020-06-03,90.99,91.62,...,NaN,Large Value,0.0725,866-909-9473,The investment seeks to track the price and yi...,[],NaN,NaN,NaN,NaN


In [10]:
bots = []
for stock in stocks:
    bots.append(create_tradebot(0, 500, stock['Name']))

In [11]:
for bot in bots:
#     finds the matching dictionary in a list of dicts
#     not sure how next functions
    match = next((stock for stock in stocks if stock['Name'] == bot.name), None)
    
    bot.dollarCostAveraging(match['Ticker'],bot.payment,begin)
    print(bot.holdings)
    

{'$': 135.83, '^GSPC': 65}
{'$': 2.04, 'VOO': 790}
{'$': 38.84, 'QQQ': 1364}
{'$': 2.08, 'SDY': 2069}
{'$': 21.05, 'VIG': 1799}
{'$': 11.98, 'KOIN': 1165}
{'$': 11.33, 'QTUM': 893}
{'$': 1512.4, '^IXIC': 28}
{'$': 2.38, 'VTSAX': 2876}
{'$': 2.56, 'AIVSX': 5345}
{'$': 6.91, 'PIODX': 7606}
{'$': 18.33, 'DVY': 2034}
{'$': 14.59, 'PFF': 4269}
{'$': 15.08, 'DTD': 2089}
{'$': 5.64, 'BLCN': 1320}
{'$': 5.13, 'LEGR': 1053}


In [12]:
# still unsure of how lambda works
bots.sort(key = lambda x: x.get_current_value(), reverse=True)

In [13]:
for bot in bots:
    match = next((stock for stock in stocks if stock['Name'] == bot.name), None)
    print(bot.name, match['Ticker'])
    print(bot.get_current_value())
    print()

nasdaq100 QQQ
322884.0

nasdaq ^IXIC
272633.88

vanguard500 VOO
226747.84

ValueHunting PIODX
226513.59000000003

vanguardTotal VTSAX
221655.69999999998

DA_nasdaq VIG
216638.63999999998

AmericanLTG AIVSX
204074.66

sp500 ^GSPC
203122.37999999998

DA_sp1500 SDY
196929.5

DA_WeightedByDividend DTD
193832.5

DA_DowJones DVY
175511.84999999998

DA_PreferredShares PFF
150624.91

BC_Economy BLCN
36622.439999999995

BC_Innovation KOIN
34414.43000000001

BC_WW LEGR
31679.37

QC_Bluestar QTUM
27122.81



In [14]:
bot.totalCashInput

22000